In [10]:
!pip install fuzzywuzzy

  Using cached fuzzywuzzy-0.18.0-py2.py3-none-any.whl (18 kB)


In [12]:
# import required packages
import pandas as pd
import numpy as np

from fuzzywuzzy import process, fuzz

In [3]:
# read the data
df = pd.read_csv('D:\\Password-Analysis\\top_200_password_2020_by_country_extended.csv')
df.head()

,country_code,country,Rank,Password,User_count,Time_to_crack,Global_rank,Time_to_crack_in_seconds,length,unique_chars,numbers,symbols,symbols_and_numbers,sequentials
0,cn,China,1,123456,8159358,< 1 second,1.0,0,6,6,6,0,6,0
1,cn,China,2,123456789,1817250,< 1 second,2.0,0,9,9,9,0,9,0
2,cn,China,3,12345678,700019,< 1 second,6.0,0,8,8,8,0,8,0
3,cn,China,4,654321,245827,< 1 second,23.0,0,6,6,6,0,6,0
4,cn,China,5,1234567890,210168,< 1 second,9.0,0,10,10,10,0,10,0


In [6]:
# get country codes
df.country_code.unique()

array(['cn', 'ru', 'es', 'us', 'vn'], dtype=object)

In [7]:
# get countrywise data
df_cn = df.loc[df['country_code'] == 'cn']
df_ru = df.loc[df['country_code'] == 'ru']
df_es = df.loc[df['country_code'] == 'es']
df_us = df.loc[df['country_code'] == 'us']
df_vn = df.loc[df['country_code'] == 'vn']

In [26]:
# defining function to calculate the fuzzy score
def fuzzy_similarity(password_df):
        """
        Function to calculate the fuzzy score between different passwords
        Args:
        password_df - 
        Return:

        """

        # calculate the fuzzy score
        fuzzy_score = [(x,) + i
                        for x in password_df 
                        for i in process.extract(x, password_df, scorer=fuzz.token_sort_ratio)]

        # create dataframe from above calculated score
        fuzzy_df = pd.DataFrame(fuzzy_score, columns=['password','match','score', 'index'])

        # drop the last column as it is not needed
        fuzzy_df.drop(columns='index', inplace=True)

        # remove the score calculation itself
        fuzzy_df = fuzzy_df[fuzzy_df['password'] != fuzzy_df['match']]

        return fuzzy_df
    

In [28]:
# calculating fuzzy score for each country
fuzzy_df_cn = fuzzy_similarity(df_cn['Password'])
fuzzy_df_ru = fuzzy_similarity(df_cn['Password'])
fuzzy_df_es = fuzzy_similarity(df_cn['Password'])
fuzzy_df_us = fuzzy_similarity(df_cn['Password'])
fuzzy_df_vn = fuzzy_similarity(df_cn['Password'])

## Performing Statistical Analysis

Null Hypothesis: We hypothesize that the passwords will be identical across different languages and countries.

$$
H_0: \mu_{China} = \mu_{United States} = \mu_{Russia} = \mu_{Mexico} = \mu_{Vietnam}
$$

Alternate Hypothesis: The password are not identical accross different languages and countries.

ANOVA is the most suitable approach for these kinds for tests, so before performing ANOVA we will first check if all the required assumptions are met or not to perform ANOVA.

ANOVA test is based on the following assumptions:
1. Data is normally distributed
2. Homogeneity of variance
3. Observation are independent of each other